In [1]:
from dotenv import load_dotenv
load_dotenv("/home/marshath/play/chainlink/algovate/.env")

Python-dotenv could not parse statement starting at line 4


True

In [2]:
import time
import pickle
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown

In [3]:
# import logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# logger = logging.getLogger(__name__)

In [4]:
from llama_index import Document
from llama_index.node_parser import SimpleNodeParser
from llama_index import GPTVectorStoreIndex, ResponseSynthesizer
from llama_index.data_structs.node import Node, DocumentRelationship

from langchain.chat_models import ChatOpenAI
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index import LLMPredictor, GPTVectorStoreIndex, PromptHelper, ServiceContext
from llama_index import GPTListIndex
from llama_index.indices.list.retrievers import ListIndexLLMRetriever
from llama_index.indices.postprocessor import (
    LLMRerank
)
from llama_index.indices.postprocessor.node import AutoPrevNextNodePostprocessor

In [11]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.indices.postprocessor.node import PrevNextNodePostprocessor
from llama_index.node_parser import SimpleNodeParser
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.storage_context import StorageContext

from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

In [12]:
# Load the data
with open('/home/marshath/play/chainlink/algovate/algovate/data/documents.pkl', 'rb') as f:
    cl_techdoc = pickle.load(f)

# convert to llama documents
llama_cl_techdoc = []
for doc in cl_techdoc:
    llama_cl_techdoc.append(Document(text=doc.page_content, extra_info=doc.metadata))

with open('/home/marshath/play/chainlink/algovate/algovate/data/combined_documents.pkl', 'rb') as f:
    cl_combineddoc = pickle.load(f)

# convert to llama documents
llama_cl_combineddoc = []
for doc in cl_combineddoc:
    llama_cl_combineddoc.append(Document(text=doc.page_content, extra_info=doc.metadata))

# load the ground truths
with open("/home/marshath/play/chainlink/algovate/algovate/data/ground_truths.pkl", "rb") as f:
    ground_truths = pickle.load(f)

df = pd.DataFrame(ground_truths)

In [13]:
# convert it to nodes
parser = SimpleNodeParser()
cl_techdoc_nodes = parser.get_nodes_from_documents(llama_cl_techdoc)
cl_combineddoc_nodes = parser.get_nodes_from_documents(llama_cl_combineddoc)

In [8]:
def retrieve_docs(df, query_engine):
    # Add retrieved_docs column if it doesn't exist
    df['retrieved_docs'] = df.get('retrieved_docs', '')

    for index, row in df.iterrows():
        retrieved_docs = query_engine.retrieve(row['question'])

        docs = ""
        for d in retrieved_docs:
            if docs == "":
                docs = d.node.text
            else:
                docs = docs + "\n\n:::NEXT DOC:::\n\n" + d.node.text

        # Store retrieved docs in the DataFrame
        df.at[index, 'retrieved_docs'] = docs

In [9]:
def evaluate_dataframe_answer(df):
    # Add right_or_wrong column if it doesn't exist
    if 'right_or_wrong' not in df.columns:
        df['right_or_wrong'] = ''

    # Widgets
    button_right = widgets.Button(description="Right")
    button_wrong = widgets.Button(description="Wrong")
    button_partial = widgets.Button(description="partial")
    out = widgets.Output()

    # Starting index
    index = [0]  # Use list to allow mutation inside functions

    def display_row(row):
        """
        Display the row data in markdown
        """
        with out:
            display(Markdown(f"**Question**: {row['question']}"))
            display(Markdown(f"**Answer**: {row['answer']}"))
            display(Markdown(f"**Result**: {row['result']}\n"))

    def on_button_right_clicked(b):
        """
        Update right_or_wrong column with 'right' and go to next row
        """
        df.at[index[0], 'right_or_wrong'] = 'right'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df):
            display_row(df.iloc[index[0]])

    def on_button_wrong_clicked(b):
        """
        Update right_or_wrong column with 'wrong' and go to next row
        """
        df.at[index[0], 'right_or_wrong'] = 'wrong'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df):
            display_row(df.iloc[index[0]])

    def on_button_partial_clicked(b):
        """
        Update right_or_wrong column with 'partial' and go to next row
        """
        df.at[index[0], 'right_or_wrong'] = 'partial'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df):
            display_row(df.iloc[index[0]])

    button_right.on_click(on_button_right_clicked)
    button_wrong.on_click(on_button_wrong_clicked)
    button_partial.on_click(on_button_partial_clicked)

    display(out)
    display(button_right, button_wrong, button_partial)

    # Start by displaying the first row
    display_row(df.iloc[0])

In [10]:
def evaluate_dataframe_retrieval(df):
    # Add rd_right_or_wrong column if it doesn't exist
    df['rd_right_or_wrong'] = df.get('rd_right_or_wrong', '')

    df_wrong = df[df['right_or_wrong'].isin(['wrong', 'partial'])].copy()

    # Widgets
    button_right = widgets.Button(description="Right")
    button_wrong = widgets.Button(description="Wrong")
    button_partial = widgets.Button(description="Partial")
    out = widgets.Output()

    # Starting index
    index = [0]  # Use list to allow mutation inside functions

    def display_row(row):
        """
        Display the row data in markdown
        """
        with out:
            display(Markdown(f"**Question**: {row['question']}"))
            display(Markdown(f"**Answer**: {row['answer']}"))
            display(Markdown(f"**Result**: {row['result']}\n"))
            display(Markdown(f"**Retrieved Docs**: {row['retrieved_docs']}"))

    def on_button_right_clicked(b):
        """
        Update right_or_wrong column with 'right' and go to next row
        """
        df.at[df_wrong.index[index[0]], 'rd_right_or_wrong'] = 'right'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df_wrong):
            display_row(df_wrong.iloc[index[0]])

    def on_button_wrong_clicked(b):
        """
        Update right_or_wrong column with 'wrong' and go to next row
        """
        df.at[df_wrong.index[index[0]], 'rd_right_or_wrong'] = 'wrong'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df_wrong):
            display_row(df_wrong.iloc[index[0]])

    def on_button_partial_clicked(b):
        """
        Update right_or_wrong column with 'partial' and go to next row
        """
        df.at[df_wrong.index[index[0]], 'rd_right_or_wrong'] = 'partial'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df_wrong):
            display_row(df_wrong.iloc[index[0]])

    button_right.on_click(on_button_right_clicked)
    button_wrong.on_click(on_button_wrong_clicked)
    button_partial.on_click(on_button_partial_clicked)

    display(out)
    display(button_right, button_wrong, button_partial)

    # Start by displaying the first row
    if len(df_wrong) > 0:
        display_row(df_wrong.iloc[0])
    else:
        print("There are no wrong rows in the DataFrame.")

### Base

In [20]:
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0,))

max_input_size = 4096
num_output = 256
max_chunk_overlap = 20

prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)


##### techdoc only

In [ ]:
index_base_techdoc = GPTVectorStoreIndex.from_documents(
    llama_cl_techdoc, 
    service_context=service_context
)

query_engine_base_techdoc = index_base_techdoc.as_query_engine()

In [25]:
df1 = df.copy()
for i, gt in df1.iterrows():
    try:
        df1.loc[i, "result"] = query_engine_base_techdoc.query(gt.question)
    except Exception as e:
        logger.error(e)
        df1.loc[i, "result"] = "ERROR"

In [26]:
retrieve_docs(df1, query_engine_base_techdoc)

In [28]:
df1.to_csv("/home/marshath/play/chainlink/algovate/test_llama/techdoc_base.csv")

##### combineddocs

In [29]:
index_base_combineddoc = GPTVectorStoreIndex.from_documents(
    llama_cl_combineddoc, 
    service_context=service_context
)

query_engine_base_combineddoc = index_base_combineddoc.as_query_engine()

In [30]:
df2 = df.copy()
for i, gt in df2.iterrows():
    try:
        df2.loc[i, "result"] = query_engine_base_combineddoc.query(gt.question)
    except Exception as e:
        logger.error(e)
        df2.loc[i, "result"] = "ERROR"

In [31]:
retrieve_docs(df2, query_engine_base_combineddoc)

In [32]:
df2.to_csv("/home/marshath/play/chainlink/algovate/test_llama/combineddoc_base.csv")

### LLMReranker

In [33]:
# based on https://medium.com/llamaindex-blog/using-llms-for-retrieval-and-reranking-23cf2d3a14b6
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, chunk_size_limit=512)
reranker = LLMRerank(choice_batch_size=5, top_n=3, service_context=service_context)

##### techdoc

In [34]:
index_llmreranker_techdoc = GPTVectorStoreIndex.from_documents(llama_cl_techdoc, service_context=service_context)

query_engine_llmreranker_techdoc = index_llmreranker_techdoc.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[reranker],
)

In [35]:
df3 = df.copy()
for i, gt in df3.iterrows():
    try:
        df3.loc[i, "result"] = query_engine_llmreranker_techdoc.query(gt.question)
    except Exception as e:
        logger.error(e)
        df3.loc[i, "result"] = "ERROR"

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 1a41b5f75d35c8bb9e19a7c96ef923bf in your message.).


In [36]:
retrieve_docs(df3, query_engine_llmreranker_techdoc)

In [37]:
df3.to_csv("/home/marshath/play/chainlink/algovate/test_llama/techdoc_llmreranker.csv")

##### combineddoc

In [38]:
index_llmreranker_combineddoc = GPTVectorStoreIndex.from_documents(llama_cl_combineddoc, service_context=service_context)

query_engine_llmreranker_combineddoc = index_llmreranker_combineddoc.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[reranker],
)

In [40]:
df4 = df.copy()
for i, gt in df4.iterrows():
    try:
        df4.loc[i, "result"] = query_engine_llmreranker_combineddoc.query(gt.question)
    except Exception as e:
        # logger.error(e)
        df4.loc[i, "result"] = "ERROR"

In [41]:
retrieve_docs(df4, query_engine_llmreranker_combineddoc)

In [42]:
df4.to_csv("/home/marshath/play/chainlink/algovate/test_llama/combineddoc_llmreranker.csv")

### AutoPrevNext

##### techdoc

In [48]:
service_context = ServiceContext.from_defaults()
nodes = service_context.node_parser.get_nodes_from_documents(llama_cl_techdoc)
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)
storage_context = StorageContext.from_defaults(docstore=docstore)

index_autopn_techdoc = GPTVectorStoreIndex(nodes, storage_context=storage_context)

node_postprocessor = AutoPrevNextNodePostprocessor(docstore=docstore, num_nodes=4, service_context=service_context)

query_engine_autopn_techdoc = index_autopn_techdoc.as_query_engine(
    similarity_top_k=2,
    node_postprocessors=[node_postprocessor],
    response_mode="tree_summarize"
)

In [49]:
df5 = df.copy()
for i, gt in df5.iterrows():
    try:
        df5.loc[i, "result"] = query_engine_autopn_techdoc.query(gt.question)
    except Exception as e:
        df5.loc[i, "result"] = "ERROR"

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorr

In [50]:
retrieve_docs(df5, query_engine_autopn_techdoc)

In [51]:
df5.to_csv("/home/marshath/play/chainlink/algovate/test_llama/techdoc_autopn.csv")

##### combineddoc

In [52]:
service_context = ServiceContext.from_defaults()

nodes = service_context.node_parser.get_nodes_from_documents(llama_cl_combineddoc)
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)
storage_context = StorageContext.from_defaults(docstore=docstore)

index_autopn_combineddoc = GPTVectorStoreIndex(nodes, storage_context=storage_context)

node_postprocessor = AutoPrevNextNodePostprocessor(docstore=docstore, num_nodes=4, service_context=service_context)

query_engine_autopn_combineddoc = index_autopn_combineddoc.as_query_engine(
    similarity_top_k=2,
    node_postprocessors=[node_postprocessor],
    response_mode="tree_summarize"
)

In [53]:
df6 = df.copy()
for i, gt in df6.iterrows():
    try:
        df6.loc[i, "result"] = query_engine_autopn_combineddoc.query(gt.question)
    except Exception as e:
        df6.loc[i, "result"] = "ERROR"

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorr

In [ ]:
retrieve_docs(df6, query_engine_autopn_combineddoc)

In [ ]:
df6.to_csv("/home/marshath/play/chainlink/algovate/test_llama/combineddoc_autopn.csv")

### Analyse

In [8]:
df = pd.read_csv("/home/marshath/play/chainlink/algovate/test_llama/techdoc_base.csv")

In [16]:
evaluate_dataframe_answer(df)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='partial', style=ButtonStyle())

In [20]:
evaluate_dataframe_retrieval(df)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='Partial', style=ButtonStyle())

In [22]:
df.drop(columns=["Unnamed: 0"], inplace=True)

In [25]:
df.to_csv("/home/marshath/play/chainlink/algovate/test_llama/techdoc_base.csv", index=False)

In [26]:
df = pd.read_csv("/home/marshath/play/chainlink/algovate/test_llama/combineddoc_base.csv")

In [27]:
evaluate_dataframe_answer(df)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='partial', style=ButtonStyle())

In [28]:
evaluate_dataframe_retrieval(df)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='Partial', style=ButtonStyle())

In [29]:
df.drop(columns=["Unnamed: 0"], inplace=True)
df.to_csv("/home/marshath/play/chainlink/algovate/test_llama/combineddoc_base.csv", index=False)

In [30]:
df = pd.read_csv("/home/marshath/play/chainlink/algovate/test_llama/techdoc_llmreranker.csv")

In [31]:
evaluate_dataframe_answer(df)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='partial', style=ButtonStyle())

In [32]:
evaluate_dataframe_retrieval(df)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='Partial', style=ButtonStyle())

In [33]:
df.drop(columns=["Unnamed: 0"], inplace=True)
df.to_csv("/home/marshath/play/chainlink/algovate/test_llama/techdoc_llmreranker.csv", index=False)

In [34]:
df = pd.read_csv("/home/marshath/play/chainlink/algovate/test_llama/combineddoc_llmreranker.csv")

In [35]:
evaluate_dataframe_answer(df)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='partial', style=ButtonStyle())

In [36]:
evaluate_dataframe_retrieval(df)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='Partial', style=ButtonStyle())

In [37]:
df.drop(columns=["Unnamed: 0"], inplace=True)
df.to_csv("/home/marshath/play/chainlink/algovate/test_llama/combineddoc_llmreranker.csv", index=False)

In [38]:
df = pd.read_csv("/home/marshath/play/chainlink/algovate/test_llama/techdoc_autopn.csv")

In [39]:
evaluate_dataframe_answer(df)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='partial', style=ButtonStyle())

In [40]:
evaluate_dataframe_retrieval(df)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='Partial', style=ButtonStyle())

In [41]:
df.drop(columns=["Unnamed: 0"], inplace=True)
df.to_csv("/home/marshath/play/chainlink/algovate/test_llama/techdoc_autopn.csv", index=False)

In [42]:
df = pd.read_csv("/home/marshath/play/chainlink/algovate/test_llama/combineddoc_autopn.csv")

In [43]:
evaluate_dataframe_answer(df)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='partial', style=ButtonStyle())

In [44]:
evaluate_dataframe_retrieval(df)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='Partial', style=ButtonStyle())

In [45]:
df.drop(columns=["Unnamed: 0"], inplace=True)
df.to_csv("/home/marshath/play/chainlink/algovate/test_llama/combineddoc_autopn.csv", index=False)

In [53]:
# list path to all dfs

df_path_list = [
    "/home/marshath/play/chainlink/algovate/test_llama/techdoc_base.csv",
    "/home/marshath/play/chainlink/algovate/test_llama/combineddoc_base.csv",
    "/home/marshath/play/chainlink/algovate/test_llama/techdoc_llmreranker.csv",
    "/home/marshath/play/chainlink/algovate/test_llama/combineddoc_llmreranker.csv",
    "/home/marshath/play/chainlink/algovate/test_llama/techdoc_autopn.csv",
    "/home/marshath/play/chainlink/algovate/test_llama/combineddoc_autopn.csv",
]

# loop through all dfs and get the average of all metrics

df_list = [pd.read_csv(df) for df in df_path_list]

for i, df in enumerate(df_list):
    print(df_path_list[i])
    print(df["right_or_wrong"].value_counts())
    print("=====================================")


/home/marshath/play/chainlink/algovate/test_llama/techdoc_base.csv
right      21
wrong       4
partial     2
Name: right_or_wrong, dtype: int64
/home/marshath/play/chainlink/algovate/test_llama/combineddoc_base.csv
right      20
wrong       4
partial     3
Name: right_or_wrong, dtype: int64
/home/marshath/play/chainlink/algovate/test_llama/techdoc_llmreranker.csv
right    24
wrong     3
Name: right_or_wrong, dtype: int64
/home/marshath/play/chainlink/algovate/test_llama/combineddoc_llmreranker.csv
right    23
wrong     4
Name: right_or_wrong, dtype: int64
/home/marshath/play/chainlink/algovate/test_llama/techdoc_autopn.csv
right      19
partial     5
wrong       3
Name: right_or_wrong, dtype: int64
/home/marshath/play/chainlink/algovate/test_llama/combineddoc_autopn.csv
right      18
partial     5
wrong       4
Name: right_or_wrong, dtype: int64


### Improving LLMRanker with AutoPrevNext

In [14]:
parser = SimpleNodeParser(
    text_splitter=RecursiveCharacterTextSplitter.from_language(
                            language=Language.MARKDOWN, 
                            chunk_size=(2000), 
                            chunk_overlap=100
    )
) 

llm_predictor = LLMPredictor(
    llm=ChatOpenAI(
        temperature=0, 
        model_name="gpt-3.5-turbo"
    )
)

service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor, 
    node_parser=parser
)

reranker = LLMRerank(
    choice_batch_size=5, 
    top_n=3, 
    service_context=service_context
)

In [15]:
nodes = service_context.node_parser.get_nodes_from_documents(
    llama_cl_techdoc
)

docstore = SimpleDocumentStore()

docstore.add_documents(
    nodes
)

storage_context = StorageContext.from_defaults(
    docstore=docstore
)

In [16]:
auto_node_postprocessor = AutoPrevNextNodePostprocessor(
    docstore=docstore, 
    num_nodes=4, 
    service_context=service_context
)


In [18]:
index = GPTVectorStoreIndex(
    nodes, 
    storage_context=storage_context
)

In [19]:
query_engine = index.as_query_engine(
    similarity_top_k=3,
    node_postprocessors=[auto_node_postprocessor, reranker],
    response_mode="compact"
)

In [20]:
df2 = df.copy()
for i, gt in df2.iterrows():
    try:
        df2.loc[i, "result"] = query_engine.query(gt.question)
    except Exception as e:
        df2.loc[i, "result"] = "ERROR"

In [22]:
evaluate_dataframe_answer(df2)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='partial', style=ButtonStyle())